In [ ]:
## import all the packages needed to run the notebook

import numpy as np
import cv2
import matplotlib.pyplot as plt
import time

from collections import deque
from imutils.video import VideoStream

import argparse

import imutils
from scipy.signal import find_peaks


## Juggle counter

In [ ]:
## create the functions needed 


# filter a specified color in an image

def ball_finder(frame, hsv_lower, hsv_upper):
    # blurr the image to reduce noise
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    #convert to hsv color space for color filtering
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    # construct a mask for the color specified
    mask = cv2.inRange(hsv, hsv_lower, hsv_upper)
    # eliminate low color signal 
    mask = cv2.erode(mask, None, iterations=2)
    # expand the strong color signal
    mask = cv2.dilate(mask, None, iterations=2)
    
    return mask
    


# count the number of peaks in a time series

def peak_counter(center_):
    # initiate a empty list to append y coordinates  
    y_coordinates = []
    # grab the y coordinates of the ball center
    for i in center_:
        y_coordinates.append(i[1])
    # get the inverse
    y_inv = np.array(y_coordinates) * (-1)
    # calculate number of peaks
    peaks, _ = find_peaks(y_inv, height=(-200, 0), distance=12)
    
    return str(len(peaks))

In [ ]:
## set the parameters

# inform the dominant color of the ball
blueLower = np.array([50,125,125])
blueUpper = np.array([165,255,255])


# specify the video path (if not using a live stream)
video_path = r'D:\Projects'
# video_path = None

In [ ]:
# initiate a list to store the center of the ball
center_ = []

# grab the right video channel
if video_path is not None:
    cap = cv2.VideoCapture(video_path)
else: 
    cap = cv2.VideoCapture(0)

while(1):
    
    time.sleep(0.025)
    
    ret, frame = cap.read()
    
    #resize the frame respecting the image ratio
    frame = imutils.resize(frame, width=800)

    # identify the ball
    mask = ball_finder(frame, blueLower, blueUpper)
    
    # find contours of the ball
    cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
 
    # proceed if at least one contour was found
    if len(cnts) > 0:
        # get the largest contour
        c = max(cnts, key=cv2.contourArea)  
        # find the center of the ball
        M = cv2.moments(c)
        # get its coordinates
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        # append them to the list
        center_.append(center)
        # calculate the number of peaks
        peaks = peak_counter(center_)
        # print the number of peaks
        cv2.putText(frame, peaks, (50,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 2)

 
    # show the frame to our screen
    cv2.imshow("Juggler Counter", frame)
    
    # Close windows with Enter
    k = cv2.waitKey(1) & 0xFF

    if k == 13:
        break
    

cap.release()
cv2.destroyAllWindows()
  

## Shadow training

In [ ]:
## step number 1, generate the shadow from a training video

In [ ]:
## set the parameters

# inform the dominant color of the ball
blueLower = np.array([50,125,125])
blueUpper = np.array([165,255,255])


# specify the path of the training video 
video_path = r'D:\Projects'
# video_path = None

In [ ]:
# initiate lists to store the coordinates and ball mask of the training session
x_ = []
y_ = []
radius_ = []
mask_ = []

# grab the right video channel
if video_path is not None:
    cap = cv2.VideoCapture(video_path)
else: 
    raise Exception('Please specify a training video')

    
while(1):
    
    time.sleep(0.05)
    
    ret, frame = cap.read()
    
    #resize the frame respecting the image ratio
    frame = imutils.resize(frame, width=800)

    # identify the ball
    mask = ball_finder(frame, blueLower, blueUpper)
    
    # save the mask for later use
    ball_train = cv2.bitwise_or(frame, frame, mask=mask)
    mask_.append(ball_train)
    
    # find contours of the ball
    cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
   # proceed if at least one contour was found
    if len(cnts) > 0:
        # get the largest contour
        c = max(cnts, key=cv2.contourArea)
        # get the coordinates to draw the circle
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        x_.append(x)
        y_.append(y)
        radius_.append(radius)
        
        # draw the circle on the frame,
        cv2.circle(frame, (int(x), int(y)), int(radius), (255, 255, 255), 2)

 
    
    
    # show the frame to our screen
    cv2.imshow("Shadow training", frame)
    
    # Close windows with Enter
    k = cv2.waitKey(1) & 0xFF

    if k == 13:
        break
    

cap.release()
cv2.destroyAllWindows()
  

In [ ]:
## step number 2, use the shadow to guide your gestures

In [ ]:
## set the parameters

# specify the path of the video (if needed) 
video_path = r'D:\Projects'
# video_path = None

In [ ]:
# grab the right video channel
if video_path is not None:
    cap = cv2.VideoCapture(video_path)
else: 
    cap = cv2.VideoCapture(0)

# initiate a counter to align the frames of the current video and training video    
count = 0
    
while(1):
    
    time.sleep(0.05)
    
    ret, frame = cap.read()
    
    # resize the frame respecting the image ratio
    frame = imutils.resize(frame, width=800)

    # draw the circle where the ball should theoretically be  
    cv2.circle(frame, (int(x_[count]), int(y_[count])), int(radius_[count]), (255, 0, 0), 2)
    
    # draw the shadow where the ball should theoretically be
    new_img = cv2.addWeighted(frame, 0.7, mask_[count], 0.3, 0)
    
    # update frame number
    count += 1
    
    # show the frame to our screen
    cv2.imshow("Shadow Training", new_img)
    
    # Close windows with Enter
    k = cv2.waitKey(1) & 0xFF

    if k == 13:
        break
    

cap.release()
cv2.destroyAllWindows()


## Augmented indoor soccer

In [ ]:
## create the decor

# import the goal image 
path = r'D:\Projects'
extension = r'\goals.png'
goals = cv2.imread(path+extension)

# resize the picture respecting the image ratio
small_goals = imutils.resize(goals, width=int(goals.shape[1]*0.6))

# create mask
goals_bw = cv2.cvtColor(small_goals, cv2.COLOR_BGR2GRAY)
mask_goal = cv2.bitwise_not(goals_bw)

# create final goal image
fg = cv2.bitwise_or(small_goals, small_goals, mask=mask_goal)

In [ ]:
## create function and set the parameters

# create a function that detect when the center of ball is within goal
def goal_validator(box,center):
    logic = box[0] < center[0] < box[2] and box[1] < center[1] < box[3]
    return logic

# specify the position of the goals 
y_x_goal = (260,390)
box = (x_y_goal[0],x_y_goal[1],x_y_goal[0]+174,x_y_goal[1]+104)



In [ ]:
cap = cv2.VideoCapture(0)


while(1):
    time.sleep(0.05)
    
    ret, frame = cap.read()
    
    #resize the frame respecting the image ratio
    frame = imutils.resize(frame, width=800)
    
    # identify the ball
    mask = ball_finder(frame, blueLower, blueUpper)
    
    # find contours of the ball
    cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
 
    # proceed if at least one contour was found
    if len(cnts) > 0:
        # get the largest contour
        c = max(cnts, key=cv2.contourArea)  
        # find the center of the ball
        M = cv2.moments(c)
        # get its coordinates
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
  
    # compile final image
    roi = frame[x_y_goal[1]:x_y_goal[1]+fg.shape[0], x_y_goal[0]:x_y_goal[0]+fg.shape[1]]
    final_roi = cv2.bitwise_or(roi, fg)
    final_image = frame.copy()
    final_image[x_y_goal[1]:x_y_goal[1]+fg.shape[0],x_y_goal[0]:x_y_goal[0]+fg.shape[1]] = final_roi
    
    # check if a goal has been scored
    if goal_validator(box, center) == True:
        final_image = cv2.putText(final_image, 'Goaaaaaal', (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 2)
        
    # show image
    cv2.imshow('Augmented indoor soccer',final_image)
    
    if cv2.waitKey(1) == 13: 
        break

cap.release()
cv2.destroyAllWindows()